In [1]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import mse, log_epoch


class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)


def start_train(dataset, in_features=1,lr = 0.1,batch_size = 32, optimizer = 'sgd'):
    start_time = time.time()
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#    print(f'Размер датасета: {len(dataset)}')
#    print(f'Количество батчей: {len(dataloader)}')

    # Создаём модель, функцию потерь и оптимизатор
    model = LinearRegression(in_features)
    criterion = nn.MSELoss()
    if optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    else:
        raise ValueError('Неизвестный оптимизатор')
    # Параметры регуляризации
    l_lambda = 0.01  # коэффициент для регуляризации
    regularization = 'l2'  # тип регуляризации l1, l2 или None

    # Параметры early stopping
    patience = 5  # количество эпох без улучшения перед остановкой
    best_loss = float('inf')
    epochs_without_improvement = 0

    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        total_loss = 0

        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred = model(batch_X)

            # Основная функция потерь
            y_pred = y_pred.squeeze()
            loss = criterion(y_pred, batch_y)

            # L2
            if regularization == 'l2':
                l2_reg = torch.tensor(0.)
                for param in model.parameters():
                    l2_reg += torch.norm(param, p=2) ** 2
                loss += l_lambda * l2_reg
            # L1
            elif regularization == 'l1':
                l1_reg = torch.tensor(0.)
                for param in model.parameters():
                    l1_reg += torch.norm(param, p=1)
                loss += l_lambda * l1_reg

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (i + 1)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            epochs_without_improvement = 0
            # Сохраняем лучшую модель
            torch.save(model.state_dict(), 'models/best_linreg_torch.pth')
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
#                print(f'Early stopping на эпохе {epoch}')
                return time.time() - start_time, avg_loss, epoch

#        if epoch % 10 == 0:
#            log_epoch(epoch, avg_loss)
    return time.time() - start_time, avg_loss, None

def evaluate():
    best_model = LinearRegression(in_features=1)
    best_model.load_state_dict(torch.load('models/best_linreg_torch.pth'))
    best_model.eval()

In [2]:
import torch
import pandas as pd
from torch.utils.data import Dataset


class CSVDataset(Dataset):
    def __init__(self, file_path, numeric_cols=None, categorical_cols=None, binary_cols=None, target_col=None):
        """
        Args:
            file_path (str): Путь к CSV файлу
            numeric_cols (list): Список числовых колонок
            categorical_cols (list): Список категориальных колонок
            binary_cols (list): Список бинарных колонок
            target_col (str): Искомое
        """
        self.data = pd.read_csv(file_path)
        self.numeric_cols = numeric_cols if numeric_cols else []
        self.categorical_cols = categorical_cols if categorical_cols else []
        self.binary_cols = binary_cols if binary_cols else []
        self.target_col = target_col

        self.preprocess_data()

    def preprocess_data(self):
        # Нормализация числовых данных (MinMax)
        for col in self.numeric_cols:
            col_data = torch.tensor(self.data[col].values, dtype=torch.float32)
            min_val = torch.min(torch.tensor(col_data))
            max_val = torch.max(torch.tensor(col_data))
            self.data[col] = ((col_data - min_val) / (max_val - min_val)) - 1

        # Кодирование категориальных данных (One-Hot), громоздкое но универсальное
        for col in self.categorical_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)
            # Конвертируем в one-hot
            one_hot = torch.zeros((len(self.data), len(unique_values)))
            one_hot[torch.arange(len(self.data)), self.data[col].values] = 1
            # Удаляем оригинальную колонку и добавляем one-hot колонки
            self.data.drop(col, axis=1, inplace=True)
            for i, val in enumerate(unique_values):
                self.data[f"{col}_{val}"] = one_hot[:, i]

        # Кодирование бинарных данных
        for col in self.binary_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = []

        # Собираем все признаки и объеденяем в один тензор с которым сможет работать Pytorch
        for col in self.numeric_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.binary_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.categorical_cols:
            for c in self.data.columns:
                if c.startswith(f"{col}_"):
                    features.append(torch.tensor(self.data.iloc[idx][c], dtype=torch.float32))

        features = torch.stack(features)

        if self.target_col:
            target = torch.tensor(self.data.iloc[idx][self.target_col], dtype=torch.float32)
            return features, target


        return features

Немного изменил код чтобы он выводил только информацию релевантную для анализа.

In [4]:
# 3.1
dataset = CSVDataset("data/Multiple.csv", numeric_cols = ["age","experience"], target_col = "income")
print('lr | batch_size | optimizer | time |    loss    | early stop epoch')
for lr in [0.01,0.1,0.2]:
    for batch_size in [16,32,64]:
        for optimizer in ['sgd','adam','rmsprop']:
            res = start_train(dataset,2,lr=lr,batch_size=batch_size,optimizer=optimizer)
            print(lr,'|   ',batch_size,'   |  ', optimizer,'  |',round(res[0],3),'|',res[1],'|',res[2])
            

C:\Users\MobS\AppData\Local\Temp\ipykernel_9112\1720545623.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_val = torch.min(torch.tensor(col_data))
C:\Users\MobS\AppData\Local\Temp\ipykernel_9112\1720545623.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_val = torch.max(torch.tensor(col_data))


lr | batch_size | optimizer | time |    loss    | early stop epoch
0.01 |    16    |   sgd   | 0.427 | 228867864.0 | 30
0.01 |    16    |   adam   | 0.089 | 1856745024.0 | 7
0.01 |    16    |   rmsprop   | 0.129 | 1621315136.0 | 10
0.01 |    32    |   sgd   | 1.167 | 177250256.0 | None
0.01 |    32    |   adam   | 1.193 | 1726917632.0 | None
0.01 |    32    |   rmsprop   | 1.202 | 1726622080.0 | None
0.01 |    64    |   sgd   | 1.187 | 177250384.0 | None
0.01 |    64    |   adam   | 1.207 | 1726946560.0 | None
0.01 |    64    |   rmsprop   | 1.194 | 1726700544.0 | None
0.1 |    16    |   sgd   | 0.322 | 81993108.0 | 25
0.1 |    16    |   adam   | 0.073 | 1758329024.0 | 6
0.1 |    16    |   rmsprop   | 0.071 | 1784133632.0 | 6
0.1 |    32    |   sgd   | 1.219 | 58795376.0 | None
0.1 |    32    |   adam   | 1.198 | 1725270784.0 | None
0.1 |    32    |   rmsprop   | 1.283 | 1723519232.0 | None
0.1 |    64    |   sgd   | 1.219 | 58795572.0 | None
0.1 |    64    |   adam   | 1.225 | 1725330

Важный вывод - с малыми батчами отлично работают adam и rmsprop

In [10]:
#3.2
import torch
import pandas as pd
from torch.utils.data import Dataset


class CSVDataset(Dataset):
    def __init__(self, file_path, numeric_cols=None, categorical_cols=None, binary_cols=None, target_col=None):
        """
        Args:
            file_path (str): Путь к CSV файлу
            numeric_cols (list): Список числовых колонок
            categorical_cols (list): Список категориальных колонок
            binary_cols (list): Список бинарных колонок
            target_col (str): Искомое
        """
        self.data = pd.read_csv(file_path)
        self.numeric_cols = numeric_cols if numeric_cols else []
        self.categorical_cols = categorical_cols if categorical_cols else []
        self.binary_cols = binary_cols if binary_cols else []
        self.target_col = target_col

        self.preprocess_data()

    def preprocess_data(self):
        # Нормализация числовых данных (MinMax)
        for col in self.numeric_cols:
            col_data = torch.tensor(self.data[col].values, dtype=torch.float32)
            min_val = torch.min(torch.tensor(col_data))
            max_val = torch.max(torch.tensor(col_data))
            self.data[col] = ((col_data - min_val) / (max_val - min_val)) - 1

        numeric_data = self.data[self.numeric_cols].values # ! добавим в датасет новый признак с средним арифметическим всех остальных
        mean_feature = torch.mean(torch.tensor(numeric_data, dtype=torch.float32), dim=1)
        self.data['mean_feature'] = mean_feature
        self.numeric_cols.append('mean_feature')
        
        squared_feature = torch.pow(torch.tensor(self.data[self.numeric_cols[0]].values, dtype=torch.float32), 2) # ! новый признак - первый численный признак в квадрате
        self.data['squared_feature'] = squared_feature
        self.numeric_cols.append('squared_feature')

        product_feature = (torch.tensor(self.data[self.numeric_cols[0]].values, dtype=torch.float32) * 
                             torch.tensor(self.data[self.numeric_cols[1]].values, dtype=torch.float32)) # ! новый признак - произведение двух численных признаков
        self.data['product_feature'] = product_feature
        self.numeric_cols.append('product_feature')


        # Кодирование категориальных данных (One-Hot), громоздкое но универсальное
        for col in self.categorical_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)
            # Конвертируем в one-hot
            one_hot = torch.zeros((len(self.data), len(unique_values)))
            one_hot[torch.arange(len(self.data)), self.data[col].values] = 1
            # Удаляем оригинальную колонку и добавляем one-hot колонки
            self.data.drop(col, axis=1, inplace=True)
            for i, val in enumerate(unique_values):
                self.data[f"{col}_{val}"] = one_hot[:, i]

        # Кодирование бинарных данных
        for col in self.binary_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = []

        # Собираем все признаки и объеденяем в один тензор с которым сможет работать Pytorch
        for col in self.numeric_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.binary_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.categorical_cols:
            for c in self.data.columns:
                if c.startswith(f"{col}_"):
                    features.append(torch.tensor(self.data.iloc[idx][c], dtype=torch.float32))

        features = torch.stack(features)

        if self.target_col:
            target = torch.tensor(self.data.iloc[idx][self.target_col], dtype=torch.float32)
            return features, target


        return features

In [ ]:
Добавил три признака в датасет, среднее арифметическое, квадрат первого, и произведение двух.

In [11]:
dataset = CSVDataset("data/Multiple.csv", numeric_cols = ["age","experience"], target_col = "income")
print('lr | batch_size | optimizer | time |    loss    | early stop epoch')
res = start_train(dataset,5)
print('0.1','|   ',32,'   |  ', 'sgd','  |',round(res[0],3),'|',res[1],'|',res[2])

C:\Users\MobS\AppData\Local\Temp\ipykernel_9112\723393383.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_val = torch.min(torch.tensor(col_data))
C:\Users\MobS\AppData\Local\Temp\ipykernel_9112\723393383.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_val = torch.max(torch.tensor(col_data))


lr | batch_size | optimizer | time |    loss    | early stop epoch
0.1 |    32    |   sgd   | 2.181 | 54616688.0 | None


По итогу с добавлением новых признаков длительность обучения выросла, но также упал loss, значит существуют ситуации,
где имеет смысл применять feature engineering.